In [ ]:

# Librerías
from imblearn.over_sampling import SMOTE  
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
os.chdir("C:/repo personal/PYTHON/deteccion de fraudes/outputs")


In [ ]:

# Cargar datos
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv").squeeze()
y_test = pd.read_csv("y_test.csv").squeeze()


In [ ]:
# Aplicar SMOTE para balancear los datos de entrenamiento

smote = SMOTE(random_state=42)  # Crear una instancia de SMOTE
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("Datos balanceados con SMOTE:")
print(f"Tamaño original: {X_train.shape}, {y_train.shape}")
print(f"Tamaño balanceado: {X_train_balanced.shape}, {y_train_balanced.shape}")

In [ ]:

# Escalar los datos
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

In [ ]:
# Crear el modelo base
model = LogisticRegression(max_iter=5000, random_state=42)
# Entrenar el modelo
model.fit(X_train_balanced, y_train_balanced)

In [ ]:
# Predicciones en el set de prueba
y_pred = model.predict(X_test)

# También obtenemos probabilidades
y_prob = model.predict_proba(X_test)[:, 1]


In [ ]:
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Clase 0", "Clase 1"], yticklabels=["Clase 0", "Clase 1"])
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


In [ ]:
# Reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
# Calcular Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)

In [ ]:
# Visualizar curva Precision-Recall
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.4f}")
plt.title("Curva Precision-Recall")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
# Imprimir métricas clave
print(f"PR-AUC: {pr_auc:.4f}")